In [4]:
import requests
import json
import pandas as pd
import geopandas as gpd
from zipfile import ZipFile
from shapely.geometry import mapping
from datetime import datetime
from functools import reduce
import numpy as np

In [2]:
sales = pd.read_csv('raw/boston_residential_sales.csv')
permits = pd.read_csv('raw/Boston Approved Building Permits 2009-2022.csv')
SAM = pd.read_csv('raw/Live_Street_Address_Management_(SAM)_Addresses.csv')

/var/folders/dp/6l162bjn4pzdgpn5hkysn1dr0000gn/T/ipykernel_73756/1451209639.py:2: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  permits = pd.read_csv('raw/Boston Approved Building Permits 2009-2022.csv')
/var/folders/dp/6l162bjn4pzdgpn5hkysn1dr0000gn/T/ipykernel_73756/1451209639.py:3: DtypeWarning: Columns (8,9,24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  SAM = pd.read_csv('raw/Live_Street_Address_Management_(SAM)_Addresses.csv')


In [3]:
#clean sales and add parcels to it - include all data, just add parcels from the SAM
SAM.rename(columns={'X': 'lon', 'Y': 'lat', 'PARCEL': 'parcel_id'}, inplace=True)
SAM_processed = SAM[['FULL_ADDRESS', 'parcel_id', 'MAILING_NEIGHBORHOOD', 'ZIP_CODE']]
SAM_processed = SAM_processed.groupby('FULL_ADDRESS').agg({'MAILING_NEIGHBORHOOD': 'first', 'parcel_id': 'first', 'ZIP_CODE': 'first'}).reset_index()
sales_parcels = pd.merge(sales, SAM_processed, left_on=['address'], right_on = ['FULL_ADDRESS'], how='left')
sales_parcels['date'] = pd.to_datetime(sales_parcels['date']).dt.date
sales_parcels = sales_parcels.sort_values(by=['parcel_id', 'date'])


In [4]:
def convert_dollar_to_float(currency_str):
    # Remove dollar signs and commas, then convert to float
    return float(currency_str.replace('$', '').replace(',', ''))

In [5]:
#clean permits
permits['issued_date'] = pd.to_datetime(permits['issued_date']).dt.date
residential_occupancy = ['1-3FAM', '6Unit', '1-2FAM', '1Unit', 'Mixed',
       '7More', '1-4FAM', 'Multi', '2unit', '1-7FAM',
       '7unit', '3unit', 'VacLd', '6unit', '4unit', '5unit', 'MIXED', '4Unit']
permits = permits[(permits['occupancytype'].isin(residential_occupancy)) | (pd.isna(permits['occupancytype']))]
permits.reset_index(drop=True, inplace=True)
permits = permits[['worktype','permittypedescr', 'description', 'comments', 'declared_valuation', 'total_fees', 'issued_date', 'owner','owner type','occupancytype','parcel_id']]
# Values to remove
values_to_remove_valu = ['($40,000.00)', '($200,000.00)']
# Create a mask for rows to drop
mask_valu_drop = permits['declared_valuation'].isin(values_to_remove_valu)
# Drop these rows
permits = permits.drop(permits[mask_valu_drop].index)
permits['declared_valuation'] = permits['declared_valuation'].apply(convert_dollar_to_float)
permits['total_fees'] = permits['total_fees'].apply(convert_dollar_to_float)

permits_process = permits.copy()


In [6]:
permits.dtypes

worktype               object
permittypedescr        object
description            object
comments               object
declared_valuation    float64
total_fees            float64
issued_date            object
owner                  object
owner type             object
occupancytype          object
parcel_id             float64
dtype: object

In [7]:
#define new df and initialize columns
sales_with_permits_by_date = sales_parcels.copy()
sales_with_permits_by_date['permit_valu_up2_sale'] = pd.Series([np.nan] * len(sales_with_permits_by_date))
sales_with_permits_by_date['permit_fees_up2_sale'] = pd.Series([np.nan] * len(sales_with_permits_by_date))
sales_with_permits_by_date['investor_permit'] = None
sales_with_permits_by_date['investor_type_permit'] = None
sales_with_permits_by_date['comments'] = None
sales_with_permits_by_date['description'] = None
sales_with_permits_by_date['permittypedescr'] = None
sales_with_permits_by_date['worktype'] = None
sales_with_permits_by_date['occupancy_permit'] = None



In [8]:
# sale = sales_with_permits_by_date.iloc[9]
# mask = (permits_process['parcel_id'] == sale['parcel_id']) & (permits_process['issued_date'] < sale['date'])

# relevant_permits = permits_process[mask]
# print(sale['parcel_id'])
# print(sale['date'])
# print(sale)
# print(relevant_permits['issued_date'])



In [9]:
def process_permits(sales_df, permits_df):

    sales_df = sales_df.sort_values(by=['parcel_id', 'date'])

    # Process each transaction
    for idx, sale in sales_df.iterrows():
        # Filter permits based on 'parcel_id' and dates prior to the sale date
        mask = (permits_df['parcel_id'] == sale['parcel_id']) & (permits_df['issued_date'] < sale['date'])
        relevant_permits = permits_df[mask]
        
        # Summarize 'declared_valuation' and 'total_fees'
        if not relevant_permits.empty:
            sales_df.at[idx, 'permit_valu_up2_sale'] = relevant_permits['declared_valuation'].sum()
            sales_df.at[idx, 'permit_fees_up2_sale'] = relevant_permits['total_fees'].sum()
            
            # Find the owner with the largest declared valuation
            largest_valuation_row = relevant_permits.loc[relevant_permits['declared_valuation'].idxmax()]

            # Get the owner and owner type from the largest valuation row
            sales_df.at[idx, 'investor_permit'] = largest_valuation_row['owner']
            sales_df.at[idx, 'investor_type_permit'] = largest_valuation_row['owner type']
            sales_df.at[idx, 'occupancy_permit'] = largest_valuation_row['occupancytype']
            
            # Concatenate and set the fields from all relevant_permits
            sales_df.at[idx, 'comments'] = '|'.join(relevant_permits['comments'].dropna().astype(str))
            sales_df.at[idx, 'description'] = ','.join(relevant_permits['description'].dropna().astype(str))
            sales_df.at[idx, 'permittypedescr'] = ','.join(relevant_permits['permittypedescr'].dropna().astype(str))
            sales_df.at[idx, 'worktype'] = ','.join(relevant_permits['worktype'].dropna().astype(str))

            # Remove processed permits from the permits_df
            permits_df = permits_df.drop(relevant_permits.index)

    return sales_df

In [10]:
updated_sales_with_permits_by_date = process_permits(sales_with_permits_by_date, permits_process)


In [11]:
updated_sales_with_permits_by_date.to_csv('/Users/adi/Dropbox (MIT)/6.C35 Group Project/Data/Modified Datasets/Adi/sales_permitsData_by_date_nullincl.csv', index=False)


0.9363216985326812

In [4]:
#continue from after making the mega dataset 
updated_sales_with_permits_by_date = pd.read_csv('output/sales_permitsData_by_date_nullincl.csv')
all_permits = pd.read_csv('raw/Boston Approved Building Permits 2009-2022.csv')

/var/folders/dp/6l162bjn4pzdgpn5hkysn1dr0000gn/T/ipykernel_67543/224187964.py:2: DtypeWarning: Columns (67,68,69,73,74,75,76,77,78,79) have mixed types. Specify dtype option on import or set low_memory=False.
  updated_sales_with_permits_by_date = pd.read_csv('output/sales_permitsData_by_date_nullincl.csv')
/var/folders/dp/6l162bjn4pzdgpn5hkysn1dr0000gn/T/ipykernel_67543/224187964.py:3: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  all_permits = pd.read_csv('raw/Boston Approved Building Permits 2009-2022.csv')


In [5]:
# Find rows in permits_df where parcel_id is NOT in sales_df
non_matching_permits = all_permits[~all_permits['parcel_id'].isin(updated_sales_with_permits_by_date['parcel_id'])]

# Append these rows to the sales_df
updated_sales_with_all_permits = pd.concat([updated_sales_with_permits_by_date, non_matching_permits], ignore_index=True)


In [7]:
updated_sales_with_all_permits['price'].isnull().sum()
updated_sales_with_all_permits.to_csv('/Users/adi/Dropbox (MIT)/6.C35 Group Project/Data/Modified Datasets/Adi/sales__and_permitsData_by_date_all.csv', index=False)


In [5]:
data = pd.read_csv("output/sales__and_permitsData_by_date_all.csv")

/var/folders/dp/6l162bjn4pzdgpn5hkysn1dr0000gn/T/ipykernel_73327/1335179767.py:1: DtypeWarning: Columns (1,4,5,8,12,13,20,21,45,46,47,48,49,50,51,52,53,55,56,62,63,67,68,69,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,91,95,96) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("output/sales__and_permitsData_by_date_all.csv")
